# DATASCI 210: Predicting The Next Pitch

Lorem ipsum delorum.

In [1]:
import psycopg2
import sys  

import numpy as np
import pandas as pd

from sklearn import cross_validation

from config import REDSHIFT_CONFIG
from src.features import *
from src.utils import *
from src.validation import *


reload(sys)
sys.setdefaultencoding('utf8')

###Connecting to RedShift 

In [2]:
# Establish a connection to the redshift database
conn = create_rs_conn(config=REDSHIFT_CONFIG)
cur = conn.cursor()

###Alan's Code Demonstrating How to Pull the Base Tables from Redshift (Don't need to run if you just want to start feature engineering...skip down to 'Sampling Pitchers')

In [3]:
# Let's see what tables are available, see ./src/utils.py for the actual underlying queries
available_tables_header, available_tables_data = get_available_rs_tables(cur)
available_tables_df = pd.DataFrame(available_tables_data)
available_tables_df.columns = available_tables_header
available_tables_df

,tablename
0,all_pitch_data
1,atbat
2,atbat_pkey
3,game
4,game_pkey
5,hitchart
6,hitchart_pkey
7,pitch
8,pitch_pkey
9,player


In [4]:
# We can run raw SQL (postgres-flavor) queries  using the run_rs_query() function from ./src/utils.py
raw_query = """select * from pitch limit 100"""
sample_header, sample_rows = run_rs_query(cur, raw_query)
sample_df = pd.DataFrame(sample_rows)
sample_df.columns = sample_header
sample_df.head()

,game_id,num,pitcher,batter,b,s,des,id,type,x,...,ay,az,break_y,break_angle,break_length,pitch_type,type_confidence,spin_dir,spin_rate,zone
0,2006/06/04/anamlb-clemlb-1,1,282332,408210,0,0,Ball,3,B,83.260,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2006/06/04/anamlb-clemlb-1,1,282332,408210,1,1,Ball,5,B,62.660,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2006/06/04/anamlb-clemlb-1,1,282332,408210,2,2,Foul,7,S,100.430,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2006/06/04/anamlb-clemlb-1,1,282332,408210,3,3,"In play, run(s)",9,X,105.580,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2006/06/04/anamlb-clemlb-1,2,282332,111851,0,1,Swinging Strike,15,S,77.250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# We can also use helper functions like get_rs_atbat_data() function from ./src/utils.py to pull_down the entire atbat data table
atbat_header, atbat_rows = get_rs_atbat_data(cur)
atbat_df = pd.DataFrame(atbat_rows)
atbat_df.columns = atbat_header

In [6]:
atbat_df.head()

,game_id,half,inning,num,b,s,o,score,batter,stand,...,des,des_es,event,event2,event3,event4,home_team_runs,away_team_runs,start_tfs,start_tfs_zulu
0,2006/06/04/anamlb-clemlb-1,top,1,2,1,3,1,None,111851,R,...,Orlando Cabrera strikes out swinging.,Orlando Cabrera se poncha tirándole.,Strikeout,None,None,None,NaN,NaN,NaN,None
1,2006/06/04/anamlb-clemlb-1,top,1,4,1,1,2,T,115223,R,...,Vladimir Guerrero homers (15) on a fly ball to...,Vladimir Guerrero batea jonrón (15) con elevad...,Home Run,None,None,None,0,2,NaN,None
2,2006/06/04/anamlb-clemlb-1,top,1,6,0,1,2,T,434778,R,...,Kendrys Morales doubles (2) on a fly ball to c...,Kendrys Morales pega doble (2) con elevado a j...,Double,None,None,None,0,3,NaN,None
3,2006/06/04/anamlb-clemlb-1,bottom,1,8,4,2,0,None,429713,L,...,Grady Sizemore walks.,Grady Sizemore recibe base por bolas.,Walk,None,None,None,NaN,NaN,NaN,None
4,2006/06/04/anamlb-clemlb-1,bottom,1,10,2,3,3,None,425509,R,...,Jhonny Peralta strikes out swinging.,Jhonny Peralta se poncha tirándole.,Strikeout,None,None,None,NaN,NaN,NaN,None


In [6]:
# The get_rs_game_data() function from ./src/utils.py retrieves the entire game data table
game_header, game_rows = get_rs_game_data(cur)
game_df = pd.DataFrame(game_rows)
game_df.columns = game_header
game_df.head()

,game_id,game_type,local_game_time,game_pk,game_time_et,home_sport_code,home_id,home_team_code,home_fname,home_sname,...,away_fname,away_sname,away_wins,away_loss,status_ind,date,day,stadium_id,stadium_name,stadium_location
0,2006/06/04/anamlb-clemlb-1,R,20:05,40780,08:05 PM,mlb,114,cle,Cleveland Indians,Cleveland,...,Los Angeles Angels,LA Angels,25,31,F,2006-06-04,SUN,5,Jacobs Field,"Cleveland, OH"
1,2006/09/16/anamlb-texmlb-1,R,19:05,42160,08:05 PM,mlb,140,tex,Texas Rangers,Texas,...,Los Angeles Angels,LA Angels,80,69,F,2006-09-16,SAT,13,Ameriquest Field in Arlington,"Arlington, TX"
2,2008/03/30/atlmlb-wasmlb-1,R,20:05,233759,08:05 PM,mlb,120,was,Washington Nationals,Washington,...,Atlanta Braves,Atlanta,0,1,F,2008-03-30,SUN,3309,Nationals Park,"Washington, DC"
3,2008/03/31/arimlb-cinmlb-1,R,14:10,233761,02:10 PM,mlb,113,cin,Cincinnati Reds,Cincinnati,...,Arizona Diamondbacks,Arizona,1,0,F,2008-03-31,MON,2602,Great American Ball Park,"Cincinnati, OH"
4,2008/04/01/anamlb-minmlb-1,R,19:10,233774,08:10 PM,mlb,142,min,Minnesota Twins,Minnesota,...,Los Angeles Angels,LA Angels,1,1,F,2008-04-01,TUE,8,Hubert H. Humphrey Metrodome,"Minneapolis, MN"


In [7]:
# The get_rs_hitchart_data() function from ./src/utils.py retrieves the entire hitchart data table
hitchart_header, hitchart_rows = get_rs_hitchart_data(cur)
hitchart_df = pd.DataFrame(hitchart_rows)
hitchart_df.columns = hitchart_header
hitchart_df.head()

,hit_id,game_id,des,x,y,batter,pitcher,type,team,inning
0,2,2014/06/06/atlmlb-arimlb-1,Groundout,150.600,156.630,518692,435221,O,A,1
1,4,2014/06/06/atlmlb-arimlb-1,Flyout,176.710,120.480,445988,527054,O,H,1
2,6,2014/06/06/atlmlb-arimlb-1,Single,106.430,83.330,600303,435221,H,A,2
3,8,2014/06/06/atlmlb-arimlb-1,Flyout,48.190,71.290,431094,527054,O,H,2
4,10,2014/06/06/atlmlb-arimlb-1,Groundout,101.410,158.630,488912,527054,O,H,2


In [8]:
# The get_rs_game_data() function from ./src/utils.py retrieves the entire player data table
player_header, player_rows = get_rs_player_data(cur)
player_df = pd.DataFrame(player_rows)
player_df.columns = player_header
player_df.head()

,team,id,pos,type,first_name,current_position,last_name,jersey_number,height,weight,bats,throws,dob
0,hou,99,--,batter,None,None,None,None,None,NaN,None,None,None
1,nyn,110189,LF,batter,Moises,None,Alou,18,6-3,230,R,R,07/03/1966
2,sfn,110383,1B,batter,Rich,PH,Aurilia,35,6-1,200,R,R,09/02/1971
3,atl,110683,P,pitcher,Miguel,None,Batista,52,6-1,210,R,R,02/19/1971
4,lan,110870,C,batter,Gary,None,Bennett,26,6-0,210,R,R,04/17/1972


###Warning: May Crash your computer!!! 

In [ ]:
# The get_rs_pitch_data() function takes a while to run, give it a few min to pull down the entire table!
pitch_header, pitch_rows = get_rs_pitch_data(cur)
pitch_df = pd.DataFrame(rows)
pitch_df.columns = header
pitch_df.head()

##Code for Sampling Pitchers from the enriched pitch data table 

In [7]:
#Get a Pandas DF of a random sample of pitch data from 5 pitchers who have each thrown at least 600 pitches
pitcher_sample_df = randomly_sample_pitchers(cursor = cur, num_pitchers = 5, min_pitch_count = 600, seed_num = 4)

In [8]:
pitcher_sample_df.head()

,game_id,num,pitcher,batter,b,s,des,id,type,x,...,home_wins,home_loss,away_id,away_fname,away_wins,away_loss,status_ind,date,day,stadium_name
0,2008/04/02/chamlb-clemlb-1,4,134320,429713,0,1,"In play, out(s)",27,X,93.560,...,2,0,145,Chicago White Sox,0,2,F,2008-04-02,WED,Progressive Field
1,2008/04/02/chamlb-clemlb-1,5,134320,452678,2,1,Called Strike,34,S,111.590,...,2,0,145,Chicago White Sox,0,2,F,2008-04-02,WED,Progressive Field
2,2008/04/02/chamlb-clemlb-1,7,134320,400098,1,0,Foul,45,S,104.720,...,2,0,145,Chicago White Sox,0,2,F,2008-04-02,WED,Progressive Field
3,2008/04/02/chamlb-clemlb-1,12,134320,434939,1,2,"In play, out(s)",88,X,109.870,...,2,0,145,Chicago White Sox,0,2,F,2008-04-02,WED,Progressive Field
4,2008/04/02/chamlb-clemlb-1,14,134320,113240,1,0,Ball,98,B,76.390,...,2,0,145,Chicago White Sox,0,2,F,2008-04-02,WED,Progressive Field
